In [5]:
from tensorflow import keras as tf
import tensorflow as T
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, LSTM
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

In [2]:
dataset = pd.read_csv('processed_train_data.csv')

X = dataset['text'].values.tolist()
Y = tf.utils.to_categorical(dataset['target'].values.tolist(), 3)

X_Test = pd.read_csv('processed_test_data.csv')['text'].values.tolist()

In [3]:
VOCAB_SIZE = 25000
MAX_LEN = 374
EMBEDDING_DIM = 128
NUM_CLASSES = 3
VOCAB_SIZE = 25000

encoder = tf.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(X)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-12-18 10:54:25.529661: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-18 10:54:25.531361: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-12-18 10:54:28.314900: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-18 10:54:28.434548: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [6]:
initializer = tf.initializers.HeNormal()

def get_model():
    model = Sequential()
    model.add(tf.layers.Input((1,), dtype=T.string))
    model.add(encoder)
    model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_LEN))
    model.add(Conv1D(filters=256, kernel_size=5, activation='relu', kernel_initializer=initializer))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu', kernel_initializer=initializer))
    model.add(Bidirectional(LSTM(units=40, dropout=0.2, return_sequences=True, kernel_initializer=initializer, recurrent_dropout=0.2)))
    model.add(Bidirectional(LSTM(units=40, dropout=0.2, kernel_initializer=initializer, recurrent_dropout=0.2)))
    model.add(Dense(40, activation='relu', kernel_initializer=initializer))
    model.add(Dense(NUM_CLASSES, kernel_initializer=initializer, activation='softmax'))

    model.compile(loss='kl_divergence', optimizer='adam', metrics=['accuracy'])
    return model

model = get_model()

/Users/praneets/Library/Python/3.9/lib/python/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [ ]:
history = model.fit(np.array(X), Y, epochs=1, batch_size=1024, validation_split=0.2)
model.save('Cnv1D_BiDirectional-LSTM_kl_1')

915/915 [==============================] - 556s 607ms/step - loss: 0.3339 - accuracy: 0.8537 - val_loss: 0.3867 - val_accuracy: 0.8293


In [ ]:
predictions = model.predict(np.array(X_Test))
preds = np.argmax(predictions, 1)
csv = 'id,target\n'
for id, pred in enumerate(preds):
  csv += '{},{}\n'.format(id, pred)
with open('submission.csv', 'w') as f:
  f.writelines(csv)

17506/17506 [==============================] - 519s 30ms/step
